## Imports

In [1]:
import pandas as pd
import folium
import os

## Data Read-in

In [11]:
df = pd.read_csv('Eviction Notices 2023.xlsx - Sheet1.csv')

## Make Map

In [14]:
df['File Date'] = pd.to_datetime(df['File Date'])

In [16]:
df = df[df['File Date'] >= '2023-01-01']

In [19]:
df['Location'].isna().value_counts()

Location
False    464
True      10
Name: count, dtype: int64

In [20]:
df = df.dropna(subset='Location')

In [21]:
def parse_location(location_str):
    # Remove the parentheses and split the string into latitude and longitude
    lat, lon = map(float, location_str[1:-1].split(','))
    return pd.Series([lat, lon], index=['latitude', 'longitude'])

# Apply the function to the 'location' column and join the result with the original DataFrame
coordinates = df['Location'].apply(parse_location)
df = df.join(coordinates)

In [22]:
pd.set_option('display.max_columns',None)

In [23]:
df['True_Count'] = df[['Non Payment', 'Breach',
       'Nuisance', 'Illegal Use', 'Failure to Sign Renewal', 'Access Denial',
       'Unapproved Subtenant', 'Owner Move In', 'Demolition',
       'Capital Improvement', 'Substantial Rehab', 'Ellis Act WithDrawal',
       'Condo Conversion', 'Roommate Same Unit', 'Other Cause',
       'Late Payments', 'Lead Remediation', 'Development']].sum(axis=1)

In [24]:
df['Eviction_Reason'] = df[['Non Payment', 'Breach',
       'Nuisance', 'Illegal Use', 'Failure to Sign Renewal', 'Access Denial',
       'Unapproved Subtenant', 'Owner Move In', 'Demolition',
       'Capital Improvement', 'Substantial Rehab', 'Ellis Act WithDrawal',
       'Condo Conversion', 'Roommate Same Unit', 'Other Cause',
       'Late Payments', 'Lead Remediation', 'Development']].idxmax(axis=1)

In [25]:
df['Eviction_Reason'].value_counts()

Eviction_Reason
Nuisance                162
Capital Improvement     116
Non Payment              66
Breach                   47
Owner Move In            38
Ellis Act WithDrawal     14
Other Cause               7
Roommate Same Unit        6
Unapproved Subtenant      4
Late Payments             2
Access Denial             2
Name: count, dtype: int64

In [27]:
def popup_html(row):
    address = row['Address']
    eviction_reason = row['Eviction_Reason']
    latitude = row['latitude']
    longitude = row['longitude']
    
    html = '''<!DOCTYPE html>
    <html>
    <strong>Address: </strong>{}'''.format(address) + '''<br>
    <strong>Eviction Reason: </strong>{}'''.format(eviction_reason) + '''<br>
    <strong>Latitude: </strong>{}'''.format(latitude) + '''<br>
    <strong>Longitude: </strong>{}'''.format(longitude) + '''<br>
    </html>
    '''
    return html

In [28]:
# Create a map object
m = folium.Map(location=[df['latitude'].mean(), df['longitude'].mean()], zoom_start=12)
# Add markers to the map
for i, row in df.iterrows():
    folium.Marker(location=[row['latitude'], row['longitude']],
                 popup=folium.Popup(popup_html(row), max_width=400)).add_to(m)
    
# Save it to an HTML file
m.save('index.html')
# Display the map
m

In [29]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/SF_Evictions2023
